In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc
import os
import re
import pymorphy2
from multiprocessing import Pool as ThreadPool
from nltk.corpus import stopwords
%matplotlib inline

Удаление стоп слов и лемматизация

In [ ]:
%%time
stopword_ru = stopwords.words('russian')
with open('preprocessing/stopwords.txt', 'r', encoding='utf-8') as f:
    for w in f.readlines():
        stopword_ru.append(w)
        
retoken = re.compile(r'[\'\w]+')
cache = {}
morph = pymorphy2.MorphAnalyzer()

def norm(word):
    '''normalize with caching'''
    if word in cache:
        return cache[word]
    nw = cache[word] = morph.parse(word)[0].normal_form
    return str(nw)

def tokenize_normalize(text):
    '''strip text to words then normalize it'''
    text = retoken.findall(str(text).lower())
    text = [norm(x) for x in text if x not in stopword_ru]
    return text

Берем датасет ленты за 2015 год.Разделяем по месяцам (интересно еще попробовать разделить по сайту и посмотреть какие топики у каких сайтов есть и как отличаются). Сохраняем отдельный текст файл для каждого лемматизированного текста в csv.

In [ ]:
def read_and_save(full_path):
    try:
        data = pd.read_csv(full_path)
        data['lemmatized_text'] = data['text'].apply(tokenize_normalize)
        for i in range(0,data.shape[0]):
            txt_arr = data['lemmatized_text'][i]
            txt = ' '.join(txt_arr)
            f_name = os.path.basename(full_path).replace(".csv","")
            new_path = os.path.dirname(full_path)+"/"+ f_name+"_" +str(i)+".txt"
            with open(new_path, "w") as file:
                file.write(txt)
    except Exception as e:
        print(e)

%%time
filenames = []
path = "/home/captcha/proj_news_viz/nlp/2015"
for dirname in os.listdir(path):
    d_path = os.path.join(path,dirname)
    for filename in os.listdir(d_path):
        filenames.append(os.path.join(d_path,filename))
        
pool = ThreadPool(6) 
results = pool.map(read_and_save, filenames)
pool.close()
pool.join() 

In [ ]:
os.chdir("/home/captcha/dynamic-nmf")
#tf-idf и построение document-term матриц
! python prep-text.py data/2015/month* -o data --tfidf --norm
#Генерация топиков для каждого временного окна
! python find-window-topics.py data/*.pkl -k 10 -o out

смотрим что получилось для рандомально выбранного значения 10

In [ ]:
! python display-topics.py -l out/month*_windowtopics_k10.pkl

In [ ]:
- Loaded model with 10 topics from out/month10_windowtopics_k10.pkl
Top 10 terms for 10 topics:
month10_01 : человек, полиция, служба, произоити, раион, дом, задержать, автомобиль, сообщить, область
month10_02 : сирия, россия, сириискии, россиискии, военныи, исламскии, сша, государство, асад, удар
month10_03 : издание, фс77, эл, допускаться, письменныи, сетевои, свидетельство, регистрация, согласие, ру
month10_04 : матч, команда, чемпионат, игра, сборныи, тренер, лига, клуб, тур, счет
month10_05 : это, которыи, страна, свои, год, россия, наш, человек, такои, один
month10_06 : рубль, год, миллиард, доллар, миллион, нефть, цена, бюджет, курс, компания
month10_07 : трансаэро, авиакомпания, аэрофлот, реис, пассажир, компания, самолет, билет, банкротство, перевозка
month10_08 : украина, newsru, ua, украинскии, год, порошенко, киев, крым, октябрь, россия
month10_09 : выбор, избирательныи, партия, выборы, избиратель, комиссия, кандидат, голосование, местныи, голос
month10_10 : отвод, днр, вооружение, донбасс, лнр, калибр, линия, донецкии, украинскии, танк

- Loaded model with 10 topics from out/month11_windowtopics_k10.pkl
Top 10 terms for 10 topics:
month11_01 : это, которыи, свои, год, человек, такои, один, наш, самыи, страна
month11_02 : египет, самолет, крушение, борт, шарм, шеих, россиискии, авиакомпания, а321, когалымавиа
month11_03 : париж, теракт, франция, террорист, французскии, серия, человек, атака, взрыв, стадион
month11_04 : полиция, служба, область, раион, задержать, сотрудник, суд, дело, уголовныи, автомобиль
month11_05 : издание, фс77, эл, допускаться, письменныи, сетевои, свидетельство, регистрация, согласие, ру
month11_06 : матч, команда, чемпионат, сборныи, игра, лига, зенит, тренер, клуб, сборная
month11_07 : антидопинговыи, wada, iaaf, допинг, спорт, федерация, атлетика, спортсмен, комиссия, вфл
month11_08 : украина, newsru, ua, украинскии, год, киев, порошенко, выбор, ноябрь, донбасс
month11_09 : рубль, год, миллиард, миллион, доллар, цена, нефть, компания, бюджет, составить
month11_10 : сирия, россия, сша, государство, сириискии, президент, исламскии, путин, военныи, иго

- Loaded model with 10 topics from out/month1_windowtopics_k10.pkl
Top 10 terms for 10 topics:
month1_01 : россия, это, которыи, страна, сша, свои, санкция, россиискии, наш, такои
month1_02 : нефть, цена, доллар, рубль, баррель, курс, евро, торг, brent, биржа
month1_03 : человек, служба, область, автомобиль, произоити, полиция, мужчина, раион, дом, водитель
month1_04 : обстрел, украинскии, донецкии, донецк, аэропорт, днр, ополченец, силовик, военныи, мариуполь
month1_05 : издание, фс77, эл, допускаться, письменныи, сетевои, свидетельство, регистрация, ру, согласие
month1_06 : charlie, hebdo, париж, франция, редакция, теракт, нападение, журнал, французскии, человек
month1_07 : матч, команда, чемпионат, клуб, сборныи, игра, кубок, турнир, счет, спорт
month1_08 : украина, встреча, порошенко, newsru, президент, ua, украинскии, киев, россия, январь
month1_09 : пользователь, содержимыи, комментарии, правило, доступ, саит, нарушать, содержание, адрес, блокировка
month1_10 : год, рубль, миллиард, миллион, компания, тысяча, составить, бюджет, декабрь, банка

- Loaded model with 10 topics from out/month2_windowtopics_k10.pkl
Top 10 terms for 10 topics:
month2_01 : это, россия, которыи, год, страна, свои, россиискии, один, такои, наш
month2_02 : украина, сша, оружие, поставка, newsru, ua, украинскии, военныи, киев, россия
month2_03 : украинскии, днр, дебальцево, обстрел, ополченец, донецкии, силовик, донбасс, донецк, раион
month2_04 : пользователь, комментарии, посетитель, саит, содержимыи, javascript, незарегистрированныи, отображение, зарегистрироваться, заслать
month2_05 : издание, фс77, эл, допускаться, письменныи, сетевои, свидетельство, регистрация, согласие, ру
month2_06 : переговоры, президент, меркель, минск, встреча, олланда, франция, путин, нормандскии, украина
month2_07 : матч, команда, клуб, чемпионат, игра, лига, динамо, сезон, зенит, игрок
month2_08 : рубль, доллар, цена, нефть, год, курс, миллиард, евро, миллион, баррель
month2_09 : год, область, служба, суд, полиция, сотрудник, человек, уголовныи, задержать, дело
month2_10 : немцов, борис, убииство, москва, мост, политика, москворецкии, оппозиционер, убитыи, следственныи

- Loaded model with 10 topics from out/month3_windowtopics_k10.pkl
Top 10 terms for 10 topics:
month3_01 : это, которыи, год, свои, человек, один, такои, самыи, наш, тот
month3_02 : cookies, to, javacript, guard, visit, turn, ddos, browser, site, have
month3_03 : украина, украинскии, донбасс, порошенко, киев, минскии, днр, донецкии, вооружение, обсе
month3_04 : рубль, миллиард, год, доллар, миллион, цена, курс, нефть, евро, составить
month3_05 : матч, команда, чемпионат, игра, сборныи, счет, динамо, лига, клуб, зенит
month3_06 : немцов, убииство, борис, дадаев, заур, дело, губашева, москва, подозревать, политика
month3_07 : область, служба, пожар, человек, полиция, уголовныи, сотрудник, пресс, задержать, раион
month3_08 : самолет, пилот, крушение, germanwings, авиакомпания, airbus, борт, разбиться, франция, a320
month3_09 : newsru, ua, год, украина, com, март, мир, февраль, тема, cсылка
month3_10 : россия, сша, президент, крым, санкция, страна, путин, россиискии, ес, глава

- Loaded model with 10 topics from out/month4_windowtopics_k10.pkl
Top 10 terms for 10 topics:
month4_01 : это, которыи, россия, год, путин, свои, президент, страна, воина, один
month4_02 : cookies, to, javacript, turn, guard, visit, ddos, browser, site, this
month4_03 : рубль, год, миллиард, доллар, миллион, курс, цена, евро, нефть, составить
month4_04 : пожар, дело, уголовныи, служба, область, суд, дом, полиция, задержать, сотрудник
month4_05 : матч, команда, чемпионат, игра, зенит, счет, цска, лига, клуб, тур
month4_06 : украина, newsru, ua, украинскии, порошенко, киев, год, ес, президент, яценюк
month4_07 : холмс, логическии, вкусныи, псевдоним, наркотик, строи, худои, любить, верить, вывод
month4_08 : украинскии, днр, донбасс, донецкии, обстрел, огонь, обсе, военныи, силовик, широкино
month4_09 : иран, сша, иемен, ядерныи, россия, санкция, страна, иранскии, соглашение, переговоры
month4_10 : непал, землетрясение, человек, траулер, самолет, дальнии, погибшии, борт, магнитуда, мчс

- Loaded model with 10 topics from out/month5_windowtopics_k10.pkl
Top 10 terms for 10 topics:
month5_01 : россия, сша, страна, президент, путин, россиискии, рф, керри, заявить, москва
month5_02 : cookies, to, javacript, turn, guard, visit, browser, support, ddos, site
month5_03 : матч, команда, чемпионат, сборныи, игра, клуб, лига, счет, сборная, зенит
month5_04 : полиция, человек, служба, дело, задержать, произоити, суд, уголовныи, область, мужчина
month5_05 : рубль, год, миллиард, миллион, доллар, компания, цена, курс, составить, нефть
month5_06 : украина, newsru, ua, порошенко, украинскии, год, киев, президент, ес, яценюк
month5_07 : фифа, президент, выбор, футбол, блаттереть, чиновник, футбольныи, коррупция, цюрих, блаттер
month5_08 : это, воина, победа, которыи, год, человек, свои, день, один, великии
month5_09 : днр, донецкии, обстрел, донбасс, украинскии, огонь, раион, луганскии, донецк, силовик
month5_10 : холмс, псевдоним, логическии, вкусныи, наркотик, строи, худои, любить, верить, женщина

- Loaded model with 10 topics from out/month6_windowtopics_k10.pkl
Top 10 terms for 10 topics:
month6_01 : россия, это, страна, сша, санкция, россиискии, которыи, путин, свои, президент
month6_02 : cookies, to, javacript, guard, turn, site, ddos, visit, browser, have
month6_03 : человек, служба, ребенок, полиция, область, раион, произоити, город, пожар, июнь
month6_04 : украина, newsru, ua, порошенко, украинскии, киев, год, президент, закон, верховныи
month6_05 : греция, кредитор, греческии, мвф, евро, еврозона, переговоры, ципраса, афина, долг
month6_06 : команда, матч, клуб, сборныи, игра, тренер, сезон, чемпионат, сборная, спорт
month6_07 : рубль, год, доллар, миллиард, миллион, компания, курс, цена, нефть, составить
month6_08 : фифа, президент, выбор, блаттереть, футбол, блаттер, чемпионат, пост, отставка, иозеф
month6_09 : пользователь, содержимыи, комментарии, правило, саит, доступ, нарушать, автор, содержание, адрес
month6_10 : обстрел, днр, украинскии, донецкии, донбасс, силовик, донецк, марьинка, раион, минскии

- Loaded model with 10 topics from out/month7_windowtopics_k10.pkl
Top 10 terms for 10 topics:
month7_01 : это, россия, которыи, год, страна, свои, россиискии, такои, один, наш
month7_02 : греция, кредитор, греческии, референдум, еврозона, страна, евро, ципраса, европеискии, афина
month7_03 : издание, фс77, эл, допускаться, письменныи, сетевои, свидетельство, регистрация, согласие, ру
month7_04 : украина, украинскии, порошенко, донбасс, сектор, правыи, киев, днр, донецкии, мукачево
month7_05 : матч, команда, чемпионат, клуб, сборныи, тренер, спорт, сезон, футбол, зенит
month7_06 : рубль, доллар, миллиард, миллион, год, нефть, курс, цена, рынок, евро
month7_07 : newsru, ua, год, com, украина, июнь, июль, мир, тема, трибунал
month7_08 : иран, ядерныи, соглашение, иранскии, сша, шестерка, переговоры, оон, санкция, программа
month7_09 : человек, служба, область, раион, произоити, полиция, пресс, сообщать, июль, сообщить
month7_10 : пользователь, содержимыи, комментарии, правило, доступ, саит, нарушать, содержание, адрес, блокировка

- Loaded model with 10 topics from out/month8_windowtopics_k10.pkl
Top 10 terms for 10 topics:
month8_01 : это, россия, страна, которыи, сша, свои, россиискии, год, президент, наш
month8_02 : рубль, доллар, нефть, курс, цена, баррель, евро, торг, биржа, валюта
month8_03 : издание, фс77, эл, допускаться, письменныи, сетевои, свидетельство, согласие, регистрация, ру
month8_04 : матч, команда, чемпионат, цска, клуб, игра, лига, спартак, тренер, тур
month8_05 : человек, взрыв, пожар, произоити, раион, погибнуть, мчс, результат, вертолет, пострадавшии
month8_06 : украина, украинскии, донбасс, киев, порошенко, днр, обстрел, военныи, донецкии, президент
month8_07 : суд, дело, васильев, уголовныи, задержать, убииство, колония, полиция, область, следствие
month8_08 : пользователь, комментарии, содержимыи, саит, правило, доступ, информация, нарушать, блокировка, адрес
month8_09 : год, миллиард, рубль, миллион, компания, тысяча, проект, составить, строительство, банка
month8_10 : newsru, год, com, ua, мир, июль, август, спорт, тема, украина

- Loaded model with 10 topics from out/month9_windowtopics_k10.pkl
Top 10 terms for 10 topics:
month9_01 : человек, раион, служба, произоити, город, сентябрь, автомобиль, пожар, дом, пресс
month9_02 : сирия, россия, сша, путин, военныи, президент, россиискии, сириискии, асад, оон
month9_03 : матч, команда, чемпионат, зенит, игра, сборныи, цска, тренер, клуб, лига
month9_04 : украина, украинскии, порошенко, киев, донбасс, ua, newsru, крым, президент, днр
month9_05 : издание, фс77, эл, допускаться, письменныи, сетевои, свидетельство, регистрация, согласие, ру
month9_06 : рубль, доллар, год, нефть, цена, миллиард, курс, миллион, рынок, баррель
month9_07 : беженец, мигрант, граница, венгрия, страна, ес, тысяча, европа, германия, евросоюз
month9_08 : выбор, партия, избирательныи, выборы, кандидат, область, голосование, депутат, единыи, губернатор
month9_09 : это, которыи, свои, год, человек, один, такои, наш, самыи, весь
month9_10 : суд, дело, уголовныи, задержать, год, следствие, преступление, прокуратура, следственныи, подозревать


### Динамическое моделирование топиков

In [ ]:
! python find-dynamic-topics.py out/month*_windowtopics_k10.pkl -k 10 -o out

Смотрим что получилось

In [ ]:
!python display-topics.py -l out/dynamictopics_k10.pkl

In [ ]:
- Loaded model with 10 topics from out/dynamictopics_k10.pkl
Top 10 terms for 10 topics:
D01 : днр, украинскии, донецкии, донбасс, обстрел, силовик, донецк, военныи, огонь, республика
D02 : матч, команда, чемпионат, игра, клуб, сборныи, лига, тренер, зенит, счет
D03 : рубль, год, доллар, миллиард, нефть, миллион, цена, курс, евро, компания
D04 : человек, служба, полиция, область, дело, произоити, суд, раион, уголовныи, задержать
D05 : это, которыи, год, свои, россия, страна, один, наш, человек, такои
D06 : издание, фс77, эл, допускаться, письменныи, сетевои, свидетельство, регистрация, согласие, ру
D07 : украина, newsru, ua, год, порошенко, украинскии, киев, com, президент, яценюк
D08 : пользователь, комментарии, содержимыи, правило, саит, доступ, автор, нарушать, информация, адрес
D09 : cookies, to, javacript, guard, turn, visit, ddos, browser, site, support
D10 : россия, сша, сирия, президент, иран, россиискии, страна, путин, военныи, государство

In [ ]:
#! python find-window-topics.py data/*.pkl -k 4,10 -o out -m out/w2v-model.bin -w selected.csv

Смотрим как динамические топики эволюционировали со временем

In [ ]:
! python track-dynamic-topics.py -l out/dynamictopics_k10.pkl out/month1_windowtopics_k10.pkl out/month2_windowtopics_k10.pkl out/month3_windowtopics_k10.pkl out/month4_windowtopics_k10.pkl out/month5_windowtopics_k10.pkl out/month6_windowtopics_k10.pkl out/month7_windowtopics_k10.pkl out/month8_windowtopics_k10.pkl out/month9_windowtopics_k10.pkl out/month10_windowtopics_k10.pkl out/month11_windowtopics_k10.pkl

In [ ]:
- Dynamic Topic: D01
Overall   : днр, украинскии, донецкии, донбасс, обстрел, силовик, донецк, военныи, огонь, республика
Window 1  : обстрел, украинскии, донецкии, донецк, аэропорт, днр, ополченец, силовик, военныи, мариуполь
Window 2  : украинскии, днр, дебальцево, обстрел, ополченец, донецкии, силовик, донбасс, донецк, раион
Window 4  : украинскии, днр, донбасс, донецкии, обстрел, огонь, обсе, военныи, силовик, широкино
Window 5  : днр, донецкии, обстрел, донбасс, украинскии, огонь, раион, луганскии, донецк, силовик
Window 6  : обстрел, днр, украинскии, донецкии, донбасс, силовик, донецк, марьинка, раион, минскии
Window 10 : отвод, днр, вооружение, донбасс, лнр, калибр, линия, донецкии, украинскии, танк
- Dynamic Topic: D02
Overall   : матч, команда, чемпионат, игра, клуб, сборныи, лига, тренер, зенит, счет
Window 1  : матч, команда, чемпионат, клуб, сборныи, игра, кубок, турнир, счет, спорт
Window 2  : матч, команда, клуб, чемпионат, игра, лига, динамо, сезон, зенит, игрок
Window 3  : матч, команда, чемпионат, игра, сборныи, счет, динамо, лига, клуб, зенит
Window 4  : матч, команда, чемпионат, игра, зенит, счет, цска, лига, клуб, тур
Window 5  : матч, команда, чемпионат, сборныи, игра, клуб, лига, счет, сборная, зенит
Window 6  : команда, матч, клуб, сборныи, игра, тренер, сезон, чемпионат, сборная, спорт
Window 7  : матч, команда, чемпионат, клуб, сборныи, тренер, спорт, сезон, футбол, зенит
Window 8  : матч, команда, чемпионат, цска, клуб, игра, лига, спартак, тренер, тур
Window 9  : матч, команда, чемпионат, зенит, игра, сборныи, цска, тренер, клуб, лига
Window 10 : матч, команда, чемпионат, игра, сборныи, тренер, лига, клуб, тур, счет
Window 11 : матч, команда, чемпионат, сборныи, игра, лига, зенит, тренер, клуб, сборная
- Dynamic Topic: D03
Overall   : рубль, год, доллар, миллиард, нефть, миллион, цена, курс, евро, компания
Window 1  : нефть, цена, доллар, рубль, баррель, курс, евро, торг, brent, биржа
Window 1  : год, рубль, миллиард, миллион, компания, тысяча, составить, бюджет, декабрь, банка
Window 2  : рубль, доллар, цена, нефть, год, курс, миллиард, евро, миллион, баррель
Window 3  : рубль, миллиард, год, доллар, миллион, цена, курс, нефть, евро, составить
Window 4  : рубль, год, миллиард, доллар, миллион, курс, цена, евро, нефть, составить
Window 5  : рубль, год, миллиард, миллион, доллар, компания, цена, курс, составить, нефть
Window 6  : рубль, год, доллар, миллиард, миллион, компания, курс, цена, нефть, составить
Window 7  : рубль, доллар, миллиард, миллион, год, нефть, курс, цена, рынок, евро
Window 8  : рубль, доллар, нефть, курс, цена, баррель, евро, торг, биржа, валюта
Window 8  : год, миллиард, рубль, миллион, компания, тысяча, проект, составить, строительство, банка
Window 9  : рубль, доллар, год, нефть, цена, миллиард, курс, миллион, рынок, баррель
Window 10 : рубль, год, миллиард, доллар, миллион, нефть, цена, бюджет, курс, компания
Window 11 : рубль, год, миллиард, миллион, доллар, цена, нефть, компания, бюджет, составить
- Dynamic Topic: D04
Overall   : человек, служба, полиция, область, дело, произоити, суд, раион, уголовныи, задержать
Window 1  : человек, служба, область, автомобиль, произоити, полиция, мужчина, раион, дом, водитель
Window 1  : charlie, hebdo, париж, франция, редакция, теракт, нападение, журнал, французскии, человек
Window 2  : год, область, служба, суд, полиция, сотрудник, человек, уголовныи, задержать, дело
Window 2  : немцов, борис, убииство, москва, мост, политика, москворецкии, оппозиционер, убитыи, следственныи
Window 3  : немцов, убииство, борис, дадаев, заур, дело, губашева, москва, подозревать, политика
Window 3  : область, служба, пожар, человек, полиция, уголовныи, сотрудник, пресс, задержать, раион
Window 3  : самолет, пилот, крушение, germanwings, авиакомпания, airbus, борт, разбиться, франция, a320
Window 4  : пожар, дело, уголовныи, служба, область, суд, дом, полиция, задержать, сотрудник
Window 4  : непал, землетрясение, человек, траулер, самолет, дальнии, погибшии, борт, магнитуда, мчс
Window 5  : полиция, человек, служба, дело, задержать, произоити, суд, уголовныи, область, мужчина
Window 6  : человек, служба, ребенок, полиция, область, раион, произоити, город, пожар, июнь
Window 7  : человек, служба, область, раион, произоити, полиция, пресс, сообщать, июль, сообщить
Window 8  : человек, взрыв, пожар, произоити, раион, погибнуть, мчс, результат, вертолет, пострадавшии
Window 8  : суд, дело, васильев, уголовныи, задержать, убииство, колония, полиция, область, следствие
Window 9  : человек, раион, служба, произоити, город, сентябрь, автомобиль, пожар, дом, пресс
Window 9  : суд, дело, уголовныи, задержать, год, следствие, преступление, прокуратура, следственныи, подозревать
Window 10 : человек, полиция, служба, произоити, раион, дом, задержать, автомобиль, сообщить, область
Window 11 : париж, теракт, франция, террорист, французскии, серия, человек, атака, взрыв, стадион
Window 11 : полиция, служба, область, раион, задержать, сотрудник, суд, дело, уголовныи, автомобиль
- Dynamic Topic: D05
Overall   : это, которыи, год, свои, россия, страна, один, наш, человек, такои
Window 1  : россия, это, которыи, страна, сша, свои, санкция, россиискии, наш, такои
Window 2  : это, россия, которыи, год, страна, свои, россиискии, один, такои, наш
Window 3  : это, которыи, год, свои, человек, один, такои, самыи, наш, тот
Window 4  : это, которыи, россия, год, путин, свои, президент, страна, воина, один
Window 5  : это, воина, победа, которыи, год, человек, свои, день, один, великии
Window 7  : это, россия, которыи, год, страна, свои, россиискии, такои, один, наш
Window 8  : это, россия, страна, которыи, сша, свои, россиискии, год, президент, наш
Window 9  : выбор, партия, избирательныи, выборы, кандидат, область, голосование, депутат, единыи, губернатор
Window 9  : это, которыи, свои, год, человек, один, такои, наш, самыи, весь
Window 10 : это, которыи, страна, свои, год, россия, наш, человек, такои, один
Window 10 : выбор, избирательныи, партия, выборы, избиратель, комиссия, кандидат, голосование, местныи, голос
Window 11 : это, которыи, свои, год, человек, такои, один, наш, самыи, страна
- Dynamic Topic: D06
Overall   : издание, фс77, эл, допускаться, письменныи, сетевои, свидетельство, регистрация, согласие, ру
Window 1  : издание, фс77, эл, допускаться, письменныи, сетевои, свидетельство, регистрация, ру, согласие
Window 2  : издание, фс77, эл, допускаться, письменныи, сетевои, свидетельство, регистрация, согласие, ру
Window 7  : издание, фс77, эл, допускаться, письменныи, сетевои, свидетельство, регистрация, согласие, ру
Window 8  : издание, фс77, эл, допускаться, письменныи, сетевои, свидетельство, согласие, регистрация, ру
Window 9  : издание, фс77, эл, допускаться, письменныи, сетевои, свидетельство, регистрация, согласие, ру
Window 10 : издание, фс77, эл, допускаться, письменныи, сетевои, свидетельство, регистрация, согласие, ру
Window 11 : издание, фс77, эл, допускаться, письменныи, сетевои, свидетельство, регистрация, согласие, ру
- Dynamic Topic: D07
Overall   : украина, newsru, ua, год, порошенко, украинскии, киев, com, президент, яценюк
Window 1  : украина, встреча, порошенко, newsru, президент, ua, украинскии, киев, россия, январь
Window 2  : украина, сша, оружие, поставка, newsru, ua, украинскии, военныи, киев, россия
Window 3  : украина, украинскии, донбасс, порошенко, киев, минскии, днр, донецкии, вооружение, обсе
Window 3  : newsru, ua, год, украина, com, март, мир, февраль, тема, cсылка
Window 4  : украина, newsru, ua, украинскии, порошенко, киев, год, ес, президент, яценюк
Window 5  : украина, newsru, ua, порошенко, украинскии, год, киев, президент, ес, яценюк
Window 6  : украина, newsru, ua, порошенко, украинскии, киев, год, президент, закон, верховныи
Window 7  : украина, украинскии, порошенко, донбасс, сектор, правыи, киев, днр, донецкии, мукачево
Window 7  : newsru, ua, год, com, украина, июнь, июль, мир, тема, трибунал
Window 8  : украина, украинскии, донбасс, киев, порошенко, днр, обстрел, военныи, донецкии, президент
Window 8  : newsru, год, com, ua, мир, июль, август, спорт, тема, украина
Window 9  : украина, украинскии, порошенко, киев, донбасс, ua, newsru, крым, президент, днр
Window 10 : украина, newsru, ua, украинскии, год, порошенко, киев, крым, октябрь, россия
Window 11 : украина, newsru, ua, украинскии, год, киев, порошенко, выбор, ноябрь, донбасс
- Dynamic Topic: D08
Overall  : пользователь, комментарии, содержимыи, правило, саит, доступ, автор, нарушать, информация, адрес
Window 1 : пользователь, содержимыи, комментарии, правило, доступ, саит, нарушать, содержание, адрес, блокировка
Window 2 : пользователь, комментарии, посетитель, саит, содержимыи, javascript, незарегистрированныи, отображение, зарегистрироваться, заслать
Window 4 : холмс, логическии, вкусныи, псевдоним, наркотик, строи, худои, любить, верить, вывод
Window 5 : холмс, псевдоним, логическии, вкусныи, наркотик, строи, худои, любить, верить, женщина
Window 6 : пользователь, содержимыи, комментарии, правило, саит, доступ, нарушать, автор, содержание, адрес
Window 7 : пользователь, содержимыи, комментарии, правило, доступ, саит, нарушать, содержание, адрес, блокировка
Window 8 : пользователь, комментарии, содержимыи, саит, правило, доступ, информация, нарушать, блокировка, адрес
- Dynamic Topic: D09
Overall  : cookies, to, javacript, guard, turn, visit, ddos, browser, site, support
Window 3 : cookies, to, javacript, guard, visit, turn, ddos, browser, site, have
Window 4 : cookies, to, javacript, turn, guard, visit, ddos, browser, site, this
Window 5 : cookies, to, javacript, turn, guard, visit, browser, support, ddos, site
Window 6 : cookies, to, javacript, guard, turn, site, ddos, visit, browser, have
- Dynamic Topic: D10
Overall   : россия, сша, сирия, президент, иран, россиискии, страна, путин, военныи, государство
Window 2  : переговоры, президент, меркель, минск, встреча, олланда, франция, путин, нормандскии, украина
Window 3  : россия, сша, президент, крым, санкция, страна, путин, россиискии, ес, глава
Window 4  : иран, сша, иемен, ядерныи, россия, санкция, страна, иранскии, соглашение, переговоры
Window 5  : россия, сша, страна, президент, путин, россиискии, рф, керри, заявить, москва
Window 5  : фифа, президент, выбор, футбол, блаттереть, чиновник, футбольныи, коррупция, цюрих, блаттер
Window 6  : россия, это, страна, сша, санкция, россиискии, которыи, путин, свои, президент
Window 6  : греция, кредитор, греческии, мвф, евро, еврозона, переговоры, ципраса, афина, долг
Window 6  : фифа, президент, выбор, блаттереть, футбол, блаттер, чемпионат, пост, отставка, иозеф
Window 7  : греция, кредитор, греческии, референдум, еврозона, страна, евро, ципраса, европеискии, афина
Window 7  : иран, ядерныи, соглашение, иранскии, сша, шестерка, переговоры, оон, санкция, программа
Window 9  : сирия, россия, сша, путин, военныи, президент, россиискии, сириискии, асад, оон
Window 9  : беженец, мигрант, граница, венгрия, страна, ес, тысяча, европа, германия, евросоюз
Window 10 : сирия, россия, сириискии, россиискии, военныи, исламскии, сша, государство, асад, удар
Window 10 : трансаэро, авиакомпания, аэрофлот, реис, пассажир, компания, самолет, билет, банкротство, перевозка
Window 11 : египет, самолет, крушение, борт, шарм, шеих, россиискии, авиакомпания, а321, когалымавиа
Window 11 : антидопинговыи, wada, iaaf, допинг, спорт, федерация, атлетика, спортсмен, комиссия, вфл
Window 11 : сирия, россия, сша, государство, сириискии, президент, исламскии, путин, военныи, иго

Поиск оптимального количества топиков  на основании когерентности ((TC-W2V)

Строим Skipgram Word2vec модель на нашем корпусе заранее лемматизированном и очищенном от стоп слов

In [ ]:
! python prep-word2vec.py data/2015 -o out -m sg

На выходе получаем  w2v-model.bin

Для каждого временного окна находим топ 3 самых оптимальных значений количества топиков основываясь на когерентности между 4 и 10 включительно

In [ ]:
! python find-window-topics.py data/*.pkl -k 4,10 -o out -m out/w2v-model.bin -w selected.csv

In [ ]:
window,k
month10,4
month11,9
month1,7
month2,7
month3,6
month4,5
month5,4
month6,7
month7,7
month8,4
month9,8

На выходе получили файл с лучшими значениями для каждого месяца

попробуем сравнить для 7 месяца:

In [ ]:
! python find-window-topics.py data/month7.pkl -k 7 -o out

In [ ]:
! python display-topics.py -l out/month7_windowtopics_k07.pkl

In [ ]:
- Loaded model with 7 topics from out/month7_windowtopics_k07.pkl
Top 10 terms for 7 topics:
month7_01 : человек, область, служба, раион, полиция, произоити, июль, пресс, дело, сообщать
month7_02 : греция, кредитор, греческии, референдум, еврозона, страна, евро, ципраса, европеискии, афина
month7_03 : издание, фс77, эл, допускаться, письменныи, сетевои, свидетельство, регистрация, согласие, ру
month7_04 : россия, это, сша, которыи, страна, иран, россиискии, год, свои, президент
month7_05 : матч, команда, чемпионат, клуб, сборныи, спорт, тренер, сезон, футбол, игрок
month7_06 : рубль, доллар, год, миллиард, миллион, нефть, цена, курс, рынок, компания
month7_07 : украина, украинскии, порошенко, newsru, донбасс, ua, киев, сектор, правыи, днр

А вот как было для 10 топиков

In [ ]:
- Loaded model with 10 topics from out/month7_windowtopics_k10.pkl
Top 10 terms for 10 topics:
month7_01 : это, россия, которыи, год, страна, свои, россиискии, такои, один, наш
month7_02 : греция, кредитор, греческии, референдум, еврозона, страна, евро, ципраса, европеискии, афина
month7_03 : издание, фс77, эл, допускаться, письменныи, сетевои, свидетельство, регистрация, согласие, ру
month7_04 : украина, украинскии, порошенко, донбасс, сектор, правыи, киев, днр, донецкии, мукачево
month7_05 : матч, команда, чемпионат, клуб, сборныи, тренер, спорт, сезон, футбол, зенит
month7_06 : рубль, доллар, миллиард, миллион, год, нефть, курс, цена, рынок, евро
month7_07 : newsru, ua, год, com, украина, июнь, июль, мир, тема, трибунал
month7_08 : иран, ядерныи, соглашение, иранскии, сша, шестерка, переговоры, оон, санкция, программа
month7_09 : человек, служба, область, раион, произоити, полиция, пресс, сообщать, июль, сообщить
month7_10 : пользователь, содержимыи, комментарии, правило, доступ, саит, нарушать, содержание, адрес, блокировка

In [ ]:
Не факт что стало лучше , но это во многом зависит от  корпуса на котором строилась word2vec модель

In [ ]:
Еще можно по такой же метрике узнать самое оптимальное количество динамических топиков

In [ ]:
! python find-dynamic-topics.py out/month1_windowtopics_k10.pkl out/month2_windowtopics_k10.pkl out/month3_windowtopics_k10.pkl  -k 4,10 -o out -m out/w2v-model.bin

In [ ]:
Top recommendations for number of dynamic topics: 5,10,6

In [ ]:
Created topic-term matrix of size 110x93714
Applying dynamic topic modeling to matrix for k=4 topics ...
Generated 110x4 factor W and 4x93714 factor H
Model coherence (k=4) = 0.4698
Saving NMF results to out/dynamictopics_k04.pkl
Applying dynamic topic modeling to matrix for k=5 topics ...
Generated 110x5 factor W and 5x93714 factor H
Model coherence (k=5) = 0.4569
Saving NMF results to out/dynamictopics_k05.pkl
Applying dynamic topic modeling to matrix for k=6 topics ...
Generated 110x6 factor W and 6x93714 factor H
Model coherence (k=6) = 0.4532
Saving NMF results to out/dynamictopics_k06.pkl
Applying dynamic topic modeling to matrix for k=7 topics ...
Generated 110x7 factor W and 7x93714 factor H
Model coherence (k=7) = 0.4407
Saving NMF results to out/dynamictopics_k07.pkl
Applying dynamic topic modeling to matrix for k=8 topics ...
Generated 110x8 factor W and 8x93714 factor H
Model coherence (k=8) = 0.4457
Saving NMF results to out/dynamictopics_k08.pkl
Applying dynamic topic modeling to matrix for k=9 topics ...
Generated 110x9 factor W and 9x93714 factor H
Model coherence (k=9) = 0.4611
Saving NMF results to out/dynamictopics_k09.pkl
Applying dynamic topic modeling to matrix for k=10 topics ...
Generated 110x10 factor W and 10x93714 factor H
Model coherence (k=10) = 0.4654
Saving NMF results to out/dynamictopics_k10.pkl
- Top recommendations for number of dynamic topics: 4,10,9